In [3]:
# python
import os, sys, pickle
from collections import OrderedDict
from itertools import combinations_with_replacement

# stats
import numpy as np
import pandas as pd
from statsmodels.api import OLS
from scipy.stats import pearsonr, spearmanr


# plot
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# neural networks
import torch, torch.utils.model_zoo # required to load nets
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

# thesis library
from lib.functions_base_analysis        import *
from lib.functions_second_analysis      import *
from lib.ImageDataset                   import ImageDataset
from lib.NetworkScorer                  import NetworkScorer
from lib.PatternGenerator               import Pattern_Generator
from lib.ActivationPattern              import Activation_Pattern
from lib.transforms                     import VisualPriorRepresentation
from lib.PatternGeneratorSearchlight    import PatternGeneratorSearchlight

DATASET_NAMES               = ('places1', 'places2', 'oasis')
SCALE_NAMES                 = ('scale2','scale4','scale8','scale16','scale32')
STUDY_NAMES                 = ("short presentation", "long presentation", "complexity order")
BEHAVIOUR_NAMES             = ('study1_places1_short.csv','study2_places1.csv','study3_places2.csv','study4_oasis.csv')

PATH_IMAGES                 = '../images and ratings/imageversions_256'
PATH_RATINGS                = '../images and ratings/ratings'
PATH_INTEGRATION_VALUES     = '../data csv/integration' # !! correlations, invert sign for integration
PATH_IB_CORRELATIONS        = '../data csv/ibcorr'
PATH_IB_CORRELATIONS_BLOCKED= '../data csv/ibcorr blocked'

PATH_RESULTS                = '../results'
PATH_PLOTS                  = '../plots'

#VisualPrior.viable_feature_tasks
MODEL_NAMES = ('autoencoding','depth_euclidean','jigsaw','reshading',
               'edge_occlusion','keypoints2d','room_layout', #'colorization' currently not working
               'curvature','edge_texture','keypoints3d','segment_unsup2d',
               'class_object','egomotion','nonfixated_pose','segment_unsup25d',
               'class_scene','fixated_pose','normal','segment_semantic',
               'denoising','inpainting','point_matching','vanishing_point')

In [4]:
#DATA_PATH = './data_256x256'
#BEHAVIOR_PATH = './behavior'
#RESULTS_PATH = './data_integration_taskonomy'

# create folder structure

In [ ]:
# create folder structure for imageversions
for dataset_name in DATASET_NAMES:
    for scale_name in SCALE_NAMES:
        for version_name in ('full','version1','version2'):
            os.makedirs(os.path.join(PATH_IMAGES, dataset_name, scale_name, version_name))

In [ ]:
# create folder structure for results
for model_name in MODEL_NAMES:
    for dataset_name in DATASET_NAMES:
        for scale_name in SCALE_NAMES:
            dir_path = os.path.join(PATH_INTEGRATION_VALUES, model_name, dataset_name, scale_name)
            os.makedirs(dir_path)

In [ ]:
# create folder strucutre for Pytorch -> Matlab converted results
for model_name in MODEL_NAMES:
    os.makedirs(os.path.join('../data mat', 'ibcorr', model_name))

for model_name in MODEL_NAMES:
    os.makedirs(os.path.join('../data mat', 'ibcorr blocked', model_name))

for model_name in MODEL_NAMES:
    os.makedirs(os.path.join('../data mat', 'integration', model_name))

In [ ]:
# create folder strucutre for Matlab -> Python converted results (analysis results)
STUDY_NAMES = ('short presentation','long presentation','complexity order','oasis')

for model_name in MODEL_NAMES:
    for study_name in STUDY_NAMES:
        for scale_name in SCALE_NAMES:
            os.makedirs(os.path.join(PATH, model_name, study_name, scale_name))


# part-whole correlations taskonomy 

In [ ]:
for model_name in MODEL_NAMES:
    print(model_name)

    # Import taskonomy model...
    VisualPriorRepresentation._load_unloaded_nets([model_name])
    net = VisualPriorRepresentation.feature_task_to_net[model_name]
    
    # ...and create activation extractor from it
    _, eval_nodes = get_graph_node_names(net)
    return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
    activation_extractor = create_feature_extractor(net, return_nodes=return_nodes)


    for dataset_name in DATASET_NAMES:
        print(dataset_name)

        for scale_name in SCALE_NAMES:
            
            dataset = ImageDataset(
                os.path.join(PATH_IMAGES, dataset_name, scale_name))
                        
            correlations, selfsimilarity, l2norm = calculate_dataset_metrics(dataset, activation_extractor)
            
            correlations.fillna(correlations.mean())
            selfsimilarity.fillna(selfsimilarity.mean())
            l2norm.fillna(l2norm.mean())
            
            correlations.to_csv(os.path.join(PATH_INTEGRATION_VALUES, model_name, dataset_name, scale_name, 'correlations.csv'), index=False, header=False)
            selfsimilarity.to_csv(os.path.join(PATH_INTEGRATION_VALUES, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)           
            l2norm.to_csv(os.path.join(PATH_INTEGRATION_VALUES, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)

## load integration

In [215]:
# load results
data_list = []


for model_name in MODEL_NAMES:
    for dataset_name in DATASET_NAMES:
        for scale_name in SCALE_NAMES:

            data = pd.read_csv(os.path.join(PATH_INTEGRATION_VALUES, model_name, dataset_name, scale_name, 'correlations.csv'), header=None)
            data.insert(0, 'scale', scale_name)
            data.insert(0, 'dataset',dataset_name)
            data.insert(0, 'model', model_name)

            data_list.append(data)
            #selfsimilarity.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)           
            #l2norm.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)

# convert correlation to integration
df = - pd.concat(data_list).set_index(['model','dataset','scale'])

In [ ]:
# block layers
df = df.T.set_index(np.insert(np.repeat(range(2,17+1), 3), 0, 1)).T.groupby(level=0,axis=1).mean()
df

In [ ]:
handles, labels = df.loc['autoencoding'].groupby('scale').mean().transpose().plot().get_legend_handles_labels()
order = [1, 3, 4, 0, 2]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], bbox_to_anchor=(1.25, 1.4))

## load ibcorr

In [ ]:
# create folder strucutre for Matlab -> Python converted results (analysis results)

for model_name in MODEL_NAMES:
    for study_name in STUDY_NAMES:
        for scale_name in SCALE_NAMES:
            os.makedirs(os.path.join('./analysis results taskonomy', model_name, study_name, scale_name))


In [256]:
#ib_list, ss_list, ibss_list = [],[],[]
ib_list = []

for model_name in MODEL_NAMES:
    for study_name in STUDY_NAMES:
        for scale_name in SCALE_NAMES:
            ib = pd.read_csv(os.path.join(PATH_IB_CORRELATIONS, model_name, study_name, scale_name, 'ib_correlations.csv'), header=None).transpose()
            ib.insert(0, 'scale', scale_name)
            ib.insert(0, 'study',study_name)
            ib.insert(0, 'model', model_name)
            ib_list.append(ib)


            # ss = pd.read_csv(os.path.join(PATH_IB_CORRELATIONS, model_name, study_name, scale_name, 'self_similarity.csv'), header=None).transpose()
            # ss.insert(0, 'scale', scale_name)
            # ss.insert(0, 'study',study_name)
            # ss.insert(0, 'model', model_name)
            # ss_list.append(ss)


            # ibss = pd.read_csv(os.path.join(PATH_IB_CORRELATIONS, model_name, study_name, scale_name, 'ib_correlation_ss_partialed.csv'), header=None).transpose()
            # ibss.insert(0, 'scale', scale_name)
            # ibss.insert(0, 'study',study_name)
            # ibss.insert(0, 'model', model_name)
            # ibss_list.append(ibss)

            #selfsimilarity.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'selfsimilarity.csv'), index=False, header=False)           
            #l2norm.to_csv(os.path.join(RESULTS_PATH, model_name, dataset_name, scale_name, 'l2norm.csv'), index=False, header=False)

ib_df = pd.concat(ib_list).set_index(['model','study','scale']).rename(columns=lambda x:x+1)
# ss_df = pd.concat(ss_list).set_index(['model','study','scale']).rename(columns=lambda x:x+1)
# ibss_df = pd.concat(ibss_list).set_index(['model','study','scale']).rename(columns=lambda x:x+1)